<img src="https://raw.github.com/emsig/logos/main/emg3d/emg3d-logo.png" width="400" alt="Logo emg3d" title="Logo emg3d">
    
### Multigrid solver for 3D EM diffusion

The modeller **emg3d** is a multigrid solver for 3D EM diffusion with tri-axial
electrical anisotropy. The matrix-free solver can be used as main solver or as
preconditioner for Krylov subspace methods, and the governing equations are
discretized on a staggered Yee grid.

- Manual: https://emg3d.emsig.xyz
- Gallery: https://emsig.xyz/emg3d-gallery/gallery/
- Code: https://github.com/emsig/emg3d
- Installation: https://emg3d.emsig.xyz/en/stable/manual/installation.html

# Simple example, comparing to empymod

## Topics

- Direct solvers versus iterative solvers
- NO MODEL BUILDER => look at, e.g., GemPy, RRM
- Computational domain is crucial
- FIT -> for complex model look at FE codes such as custEM, PETGEM

In [ ]:
# Uncomment on Google Colab
# %pip install emg3d pooch matplotlib xarray h5py discretize tqdm scooby

In [ ]:
import emg3d
import empymod
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

plt.style.use('bmh')

%matplotlib widget

## 1D Model, compute with empymod

In [ ]:
# Survey parameters
src = (-100, 100, 0, 0, -2950, -2950)
off = np.linspace(500, 6000, 101)
rec = (off, off*0, -3000, 0, 0)
frequency = 0.5

# Model parameters
depth = [0, -3000, -4000, -4200]

# Collect
inp = {
    'src': src,
    'rec': rec,
    'depth': depth,
    'freqtime': frequency,
    'srcpts': 5,
    'verb': 1,
}

# Compute responses with empymod
resp1 = empymod.bipole(res=[2e14, 0.3, 1, 1, 1], **inp)    # Background
resp2 = empymod.bipole(res=[0.3, 0.3, 1, 1, 1], **inp)     # Background, without air
resp3 = empymod.bipole(res=[2e14, 0.3, 1, 100, 1], **inp)  # Target

# Plot it
fig1, (ax11, ax12) = plt.subplots(1, 2, figsize=(10, 5), sharex=True)

fig1.suptitle("1D Model with empymod; testing air influence")
ax11.plot(off, resp1.amp(), 'k', label='background')
ax11.plot(off, resp2.amp(), ':', label='without air')
ax11.plot(off, resp3.amp(), 'C1--', label='target')
ax11.set_yscale('log')
ax11.legend()

ax12.plot(off, resp1.pha(deg=True), 'k', label='background')
ax12.plot(off, resp2.pha(deg=True), ':', label='without air')
ax12.plot(off, resp3.pha(deg=True), 'C1--', label='target')

## emg3d

In [ ]:
emg3d.meshes.good_mg_cell_nr()

In [ ]:
hx = np.ones(80)*200
hy = np.ones(32)*400
hz = np.ones(40)*200
grid = emg3d.TensorMesh([hx, hy, hz], origin=(-5100, -6600, -8000))
grid

## Create 3D layered model

In [ ]:
res = np.ones(grid.shape_cells)
res[:, :, grid.cell_centers_z > -3000] = 0.3
res[:, :, (grid.cell_centers_z > -4200) & (grid.cell_centers_z < -4000)] = 100

In [ ]:
model = emg3d.Model(grid, property_x=res, mapping='Resistivity')
model

In [ ]:
grid.plot_3d_slicer(model.property_x, pcolor_opts={'norm': LogNorm(vmin=0.3, vmax=100)})

## Generate the source field

In [ ]:
sfield = emg3d.fields.get_source_field(grid, source=src, frequency=frequency)
sfield = emg3d.fields.get_source_field(grid, source=(0, 0, -2950, 0, 0), frequency=frequency)
grid.plot_3d_slicer(sfield.fx.ravel('F'), view='abs', v_type='Ex',)

## Solve for the electric field

In [ ]:
efield = emg3d.solve(model, sfield, verb=4)

In [ ]:
grid.plot_3d_slicer(
        efield.fx.ravel('F'), view='abs', v_type='Ex',
        pcolor_opts={'norm': LogNorm(vmin=1e-15)}
)

## Get responses and compare

In [ ]:
resp4 = efield.get_receiver((off, off*0, -3000, 0, 0), 'linear')

In [ ]:
# Plot it
fig2, (ax21, ax22) = plt.subplots(1, 2, figsize=(10, 5), sharex=True)

fig2.suptitle("1D Model comparing empymod and emg3d")

ax21.plot(off, resp1.amp(), 'k', label='background')
ax21.plot(off, resp3.amp(), 'C0-', label='target empymod')
ax21.plot(off, resp4.amp(), 'C1--', label='target emg3d')
ax21.set_yscale('log')
ax21.legend()

ax22.plot(off, resp1.pha(deg=True), 'k')
ax22.plot(off, resp3.pha(deg=True), 'C0-')
ax22.plot(off, resp4.pha(deg=True), 'C1--')

## Actual 3D target

In [ ]:
res2 = np.ones(grid.shape_cells)
res2[:, :, grid.cell_centers_z > -3000] = 0.3
res2 = res2.ravel('F')

xx = (grid.cell_centers[:, 0] >= 0) & (grid.cell_centers[:, 0] <= 6000)
yy = abs(grid.cell_centers[:, 1]) <= 500
zz = (grid.cell_centers[:, 2] > -4200)*(grid.cell_centers[:, 2] < -4000)

res2[xx*yy*zz] = 100.  # Target resistivity

model2 = emg3d.Model(grid, property_x=res2, mapping='Resistivity')

In [ ]:
grid.plot_3d_slicer(model2.property_x, pcolor_opts={'norm': LogNorm(vmin=0.3, vmax=100)})

In [ ]:
sfield = emg3d.fields.get_source_field(grid, source=(0, 0, -2950, 0, 0), frequency=frequency)
efield2 = emg3d.solve(model2, sfield, verb=2)

In [ ]:
resp5 = efield2.get_receiver((off, off*0, -3000, 0, 0), 'linear')

In [ ]:
# Plot it
fig3, (ax31, ax32) = plt.subplots(1, 2, figsize=(10, 5), sharex=True)

fig3.suptitle("1D vs 3D Model")

ax31.plot(off, resp1.amp(), 'k', label='background')
ax31.plot(off, resp3.amp(), 'C0-', label='1D target empymod')
ax31.plot(off, resp5.amp(), 'C1--', label='3D target emg3d')
ax31.set_yscale('log')
ax31.legend()

ax32.plot(off, resp1.pha(deg=True), 'k')
ax32.plot(off, resp3.pha(deg=True), 'C0-')
ax32.plot(off, resp5.pha(deg=True), 'C1--')

In [ ]:
grid.plot_3d_slicer(
        efield2.fx.ravel('F'), view='abs', v_type='Ex',
        pcolor_opts={'norm': LogNorm(vmin=1e-15)}
)

In [ ]:
emg3d.Report()